### 生産最適化

問題：原料に限りがある状態で製品を生産して、利益を最大化したい


諸元の読み込み：まず製品ごとの原料価格と利益、減量ごとの在庫をファイルから読み込む

In [4]:
import pandas as pd
from pulp import lpSum, value
from ortoolpy import model_max, addvars, addvals

In [6]:
df0=pd.read_csv('prod_cost.csv',index_col=0)
df0

,原料1,原料2,原料3,利益
製品1,1,4,3,5.0
製品2,2,4,1,4.0
在庫,40,80,50,NaN


#### 表の分離
製品ごとの表：df
在庫：inv

In [8]:
df,inv=df0.iloc[:-1,:].copy(), df0.iloc[-1,:-1]
addvars(df)
df

,原料1,原料2,原料3,利益,Var
製品1,1,4,3,5.0,v000003
製品2,2,4,1,4.0,v000004


In [9]:
inv

原料1    40.0
原料2    80.0
原料3    50.0
Name: 在庫, dtype: float64

#### 実行

In [10]:
m = model_max()
m += lpSum(df.利益*df.Var) # 純利益を表す目的関数
for item in df.columns[:-2]: # 製品ごとの処理
    # 制約条件：原料の使用量が在庫以下
    m += lpSum(df[item] * df.Var) <= inv[item]
m.solve()
value(m.objective)

95.0

In [11]:
addvals(df)
df

,原料1,原料2,原料3,利益,Var,Val
製品1,1,4,3,5.0,v000003,15.0
製品2,2,4,1,4.0,v000004,5.0
